# PRUEBA CQR-d

In [1]:
# Biblioteca para aprendizaje profundo
import torch
import torchvision

# 
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F

#
if not torch.cuda.is_available():
    raise RuntimeError(
        "CUDA no está disponible. PyTorch no reconoce la GPU."
    )
device = "cuda"

In [2]:
import os
working_dir = os.getcwd()
print(working_dir)
data_dir = working_dir + "/data/AE_maxillofacial/preprocessed/"

/mnt/homeGPU/dgonzalez/tfg-bioprofile-uncertainty


In [3]:
# Manejo del sistema
import sys
sys.path.append(os.path.abspath("src/"))

In [4]:
# Manejo de argumentos de línea de comandos
import argparse

# Control de advertencias
import warnings

# Manipulación de datos
import numpy as np
import pandas as pd

# Manejo y edición de imágenes
from PIL import Image

# Visualización de datos
import matplotlib.pyplot as plt
import seaborn as sns

# Operaciones aleatorias
import random

# Funciones matemáticas avanzadas
import math

# Evaluación y partición de modelos
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# Manejo de tiempo y fechas
import time

# Modelos y funciones de pérdida personalizados 
from custom_models import ResNeXtRegressor, QuantileLoss
from coverage_metrics import empirical_coverage, mean_interval_size

In [5]:
# Creamos una semilla de aleatoriedad 
SEED = 23

# Fija la semilla para las operaciones aleatorias en Python puro
random.seed(SEED)

# Fija la semilla para las operaciones aleatorias en NumPy
np.random.seed(SEED)

# Fija la semilla para los generadores aleatorios de PyTorch en CPU
torch.manual_seed(SEED)

# Fija la semilla para todos los dispositivos GPU (todas las CUDA devices)
torch.cuda.manual_seed_all(SEED)

# Desactiva la autooptimización de algoritmos en cudnn, que puede introducir no determinismo
# torch.backends.cudnn.benchmark = False

# Fuerza a cudnn a usar operaciones determinísticas (más lento pero reproducible)
# torch.backends.cudnn.deterministic = True

# Obliga a Pytorch a usar algoritmos determinísticos cuando hay alternativa. Si no la hay, lanza un error.
# torch.use_deterministic_algorithms(True)

# Función auxiliar para asegurar que cada worker de DataLoader use una semilla basada en la global
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

# Generador de números aleatorios para DataLoader
g = torch.Generator()
g.manual_seed(SEED)

In [6]:
# Define las transformaciones aplicadas a las imágenes durante el entrenamiento en cada época.
# Estas transformaciones son aleatorias dentro de los rangos especificados, por lo que varían en cada época.
# - Redimensiona las imágenes a 448x224. Se ha escogido este tamaño dado que las imágenes son panorámicas y 
#   bastante maś anchas que altas.
# - (Regularización) Realiza un volteo horizontal a la mitad de las imágenes.
# - (Regularización) Aplica una rotación aleatoria de hasta +/-3 grados.
# - (Regularización) Aplica una transformación afín aleatoria con ligeras traslaciones (2%) y escalado (entre 
#   95% y 105%).
# - (Regularización) Modifica aleatoriamente el brillo y contraste para simular condiciones de iluminación 
#   variables.
# - Convierte la imagen a tensor, para que pueda ser manipulada por PyTorch.
# - Normaliza para ajustar la media y desviación típica de los canales RGB a los valores usados durante el 
#   entrenamiento en ImageNet.
train_transform = transforms.Compose(
    [transforms.Resize((448, 224)),
     transforms.RandomHorizontalFlip(p=0.5),
     transforms.RandomRotation(degrees=3),
     transforms.RandomAffine(degrees=0, translate=(0.02, 0.02), scale=(0.95, 1.05)), 
     transforms.ColorJitter(brightness=0.1, contrast=0.1), 
     transforms.ToTensor(),
     transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))]
)

# Define las transformaciones para las imágenes de validación y test, que son iguales que para entrenamiento 
# pero sin regularización
valid_transform = test_transform = transforms.Compose(
    [transforms.Resize((448, 224)),
     transforms.ToTensor(),
     transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))]
) 

In [7]:
class MaxillofacialXRayDataset(Dataset):
    
    def __init__(self, metadata_file, images_dir, transform=None):
        """
        metadata_file: Ruta al fichero de metadatos (CSV u otro formato)
        images_dir: Ruta al directorio de imágenes (entrenamiento o prueba)
        transform: Transformaciones a aplicar a las imágenes (normalización, etc.)
        """
        self.metadata = pd.read_csv(metadata_file)  # Cargar los metadatos
        self.images_dir = images_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.metadata)
        
    def __getitem__(self, idx):
        # Obteniene el nombre de la imagen y su valor desde los metadatos
        img_name = os.path.join(self.images_dir, self.metadata.iloc[idx]['ID'])  # Ajusta según la estructura
        target = self.metadata.iloc[idx]['Age'].astype(np.float32)  # Ajusta según el formato de tus metadatos
        
        # Abre la imagen
        image = Image.open(img_name)
        
        # Aplica transformaciones si es necesario
        if self.transform:
            image = self.transform(image)
        
        return image, target

In [8]:
# Crea el Dataset de entrenamiento con augmentations
trainset = MaxillofacialXRayDataset(
    metadata_file=data_dir + 'metadata_train.csv',
    images_dir=data_dir + 'train/',
    transform=train_transform
)

# Crea el Dataset de validación con solo resize y normalización 
validset = MaxillofacialXRayDataset(
    metadata_file=data_dir + 'metadata_train.csv',  
    images_dir=data_dir + 'train/',               
    transform=valid_transform                       
)

# Crea el Dataset de test con solo resize y normalización
testset  =  MaxillofacialXRayDataset(
    metadata_file = data_dir + 'metadata_test.csv',
    images_dir = data_dir + 'test/',
    transform = test_transform
) 


In [9]:
# Establece un batch size de 32 
BATCH_SIZE = 32

# Obtiene las edades enteras del trainset
intAges = np.floor(trainset.metadata['Age'].astype(float).to_numpy()).astype(int)
# Hay una única instancia con edad 26, que el algoritmo de separación de entrenamiento y validación será 
# incapaz de dividir de forma estratificada. Para evitar el error, reasigna esa instancia a la edad 
# inmediatamente inferior
intAges[intAges==26]=25

# Función auxiliar para crear un DataLoader a partir de un subconjunto del dataset
def create_loader(dataset, indices):
    subset = Subset(dataset, indices)
    return DataLoader(subset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, 
                      pin_memory=True, worker_init_fn=seed_worker, generator=g)
 

# Divide el conjunto de datos completo de entrenamiento en tres subconjuntos de forma estratificada:
# - Entrenamiento (68% de las instancias)
# - Validación (17% de las instancias)
# - Calibración (15% de las instancias)

train_idx, calib_idx = train_test_split(
    range(len(trainset)), train_size=0.85, shuffle=True, stratify=intAges
)

train_idx, valid_idx = train_test_split(
    train_idx, train_size=0.8, shuffle=True, stratify=[intAges[i] for i in train_idx]
)

train_loader = create_loader(trainset, train_idx)
valid_loader = create_loader(validset, valid_idx)
calib_loader = create_loader(validset, calib_idx)

# Crea DataLoader de test
test_loader = DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False)

In [10]:
alpha = 0.1

# Inicializa el modelo con múltiples salidas, una por cada cuantil
model = ResNeXtRegressor().to(device)

#
checkpoint = torch.load("models/AE_model03_splitCP_90%.pth", weights_only=False)

# Carga los pesos del modelo 
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [11]:
def inference(
    model, dataloader, device='cuda', 
    return_targets=False, return_outputs=True, return_features=False
):
    # Pone la red en modo evaluación 
    model.eval()
    
    # Inicializa listas si son requeridas
    all_targets = [] if return_targets else None
    all_outputs = [] if return_outputs else None
    all_features = [] if return_features else None

    with torch.no_grad():
        for batch in dataloader:
            # Soporta tanto (inputs, targets) como solo inputs
            if isinstance(batch, (list, tuple)) and len(batch) == 2:
                inputs, targets = batch
                inputs = inputs.to(device)
                if return_targets:
                    all_targets.append(targets.cpu())
            else:
                inputs = batch
                inputs = inputs.to(device)

            # Modelado según los flags
            if return_features and return_outputs:
                features, outputs = model(inputs, mode='both')
                all_features.append(features.cpu())
                all_outputs.append(outputs.cpu())
            elif return_features:
                features = model(inputs, mode='features')
                all_features.append(features.cpu())
            elif return_outputs:
                outputs = model(inputs)
                all_outputs.append(outputs.cpu())

    # Concatena según sea necesario
    result = []
    if return_targets:
        result.append(torch.cat(all_targets))
    if return_features:
        result.append(torch.cat(all_features))
    if return_outputs:
        result.append(torch.cat(all_outputs))

    # Si solo hay un resultado, lo devuelve directamente
    return result[0] if len(result) == 1 else tuple(result)

In [12]:
# Obtiene valores verdaderos, características extraídas y valores predichos del conjunto de validación
valid_true_values, valid_features, valid_pred_values = \
    inference(model, valid_loader, return_targets=True, return_features=True)

valid_errors = torch.abs(valid_true_values - valid_pred_values)

from sklearn.ensemble import RandomForestRegressor

X = valid_features.numpy()
y = valid_errors.numpy()

error_model = RandomForestRegressor(n_estimators=100, random_state=SEED)

error_model.fit(X, y)

,n_estimators,100
,criterion,'squared_error'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [13]:
# Obtener predicciones y valores verdaderos del conjunto de calibración
calib_true_values, calib_features, calib_pred_values = \
    inference(model, calib_loader, return_targets=True, return_features=True)

In [14]:
# Calcula el nivel de cuantificación ajustado basado en el tamaño del conjunto de calibración y alpha
n = len(calib_true_values)
q_level = np.ceil((1-alpha) * (n + 1)) / n

sigma_hat_calib = error_model.predict(calib_features.numpy())
sigma_hat_calib = np.clip(sigma_hat_calib, 1e-6, None)  # evitar divisiones por cero


calib_scores = np.abs(calib_true_values.numpy() - calib_pred_values.numpy()) / sigma_hat_calib

# Cuantil para el intervalo
q_hat = np.quantile(calib_scores, q_level, method='higher')

In [19]:
# Obtiene los valores predichos y los verdaderos 
test_true_values, test_features, test_pred_values = \
    inference(model, test_loader, return_targets=True, return_features=True)

In [ ]:
sigma_hat_test = error_model.predict(test_features.numpy())
sigma_hat_test = np.clip(sigma_hat_test, 1e-6, None)

test_pred_lower_bound = test_pred_values - q_hat * sigma_hat_test
test_pred_upper_bound = test_pred_values + q_hat * sigma_hat_test


print("Métricas de las predicción puntual:")

# Calcula el MAE y lo imprime
test_mae = torch.mean(torch.abs(test_true_values - test_pred_values))
print(f"- Error Absoluto Medio (MAE) en test: {test_mae:.3f}")

# Calcula e imprime el MSE y lo imprime
test_mse = torch.mean((test_true_values - test_pred_values) ** 2)
print(f"- Error Cuadrático Medio (MSE) en test: {test_mse:.3f}")

print("Métricas de la predicción interválica:")
    
# Calcula la cobertura empírica y lo imprime
empirical_coverage = empirical_coverage(test_pred_lower_bound, test_pred_upper_bound, test_true_values)
print(f"- Cobertura empírica (para {(1-alpha)*100}% de confianza): {empirical_coverage*100:>6.3f} %")

# Calcula el tamaño medio del intervalo de predicción y lo imprime
mean_interval_size = mean_interval_size(test_pred_lower_bound, test_pred_upper_bound)
print(f"- Tamaño medio del intervalo: {mean_interval_size:>5.3f}")

Métricas de las predicción puntual:
- Error Absoluto Medio (MAE) en test: 1.196
- Error Cuadrático Medio (MSE) en test: 2.446
- Cobertura empírica (para 90.0% de confianza): 89.916 %
- Tamaño medio del intervalo: 4.795


/tmp/ipykernel_2232/2695004989.py:4: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  test_pred_lower_bound = test_pred_values - q_hat * sigma_hat_test
/tmp/ipykernel_2232/2695004989.py:5: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  test_pred_upper_bound = test_pred_values + q_hat * sigma_hat_test


In [ ]:
new_loader = test_loader

# Obtiene valores verdaderos, características extraídas y valores predichos del conjunto de calibración 
calib_true_values, calib_features, calib_pred_values = \
    inference(model, calib_loader, return_targets=True, return_features=True)

# Obtener características extraídas y valores predichos de las nuevas instancias
new_true_values, new_features, new_pred_values = \
    inference(model, new_loader, return_targets=True, return_features=True)

In [116]:
k=200

# Calcula el nivel de cuantificación ajustado basado en el tamaño del conjunto de calibración y alpha
n = len(calib_true_values)
# global_q_level = np.ceil((1-alpha/2) * (n + 1)) / n
global_q_level = 1.0 - alpha / 2.0

# Calcula las conformity scores para los límites inferior y superior 
global_calib_scores_lower = calib_pred_values[:, 0] - calib_true_values # diferencia entre predicción inferior y valor real
global_calib_scores_upper = calib_true_values - calib_pred_values[:,-1] # diferencia entre valor real y predicción superior

# Calcula los cuantiles para ambos límites del intervalo predictivo
global_quantile_lower = np.quantile(global_calib_scores_lower, global_q_level, method='higher')
global_quantile_upper = np.quantile(global_calib_scores_upper, global_q_level, method='higher')

In [117]:
mean = calib_features.mean(dim=0, keepdim=True)
std = calib_features.std(dim=0, keepdim=True)
eps = 1e-6
std = std.clamp_min(eps)

new_features_std = (new_features - mean) / std
calib_features_std = (calib_features - mean) / std

dists = torch.cdist(new_features_std, calib_features_std, p=2)

In [118]:
new_norm = F.normalize(new_features_std, p=2, dim=1)
calib_norm = F.normalize(calib_features_std, p=2, dim=1)

cos_sim = torch.matmul(new_norm, calib_norm.T)  # (N, M)
dists = 1 - cos_sim  # (N, M)

In [119]:
# new_norm = F.normalize(new_features, p=2, dim=1)
# calib_norm = F.normalize(calib_features, p=2, dim=1)
# # cos_sim = torch.matmul(new_norm, calib_norm.T)  # (N, M)
# # dists = 1 - cos_sim  # (N, M)
# dists = torch.cdist(new_norm, calib_norm, p=2)



# # Calcula las distancias entre las features de las nuevas instancias y las de calibración [N,M]
# dists = torch.cdist(new_features, calib_features, p=2)

# Obtiene los k vecinos en calibración más cercanos a cada instancia nueva [N,k]
topk_dists, topk_idxs = torch.topk(dists, k=k, largest=False)

# Calcula la densidad local como el inverso de la distancia media a los k vecinos más cercanos [N]
local_density = 1.0 / topk_dists.mean(dim=1) # La densidad es mayor cuanto más cerca están los vecinos (0, +inf)

# Calcula el peso local a partir de la densidad, mapeado entre 0 y 1 [N]
local_weight = 1.0 - 1.0 / (1.0 + local_density) # El peso se acerca más a 0 cuanto más distintes están los vecinos, y más a 1 cuanto más cerca están

# Calcula el peso global [N]
global_weight = 1.0 - local_weight

In [120]:
# Calcula el nivel de cuantificación ajustado basado en el tamaño del conjunto de vecinos y alpha
# local_q_level = np.ceil( (1.0 - alpha/2) * (k + 1) ) / k
local_q_level = 1.0 - alpha / 2.0

# Calcula las conformity scores para los límites inferior y superior en cada vecindario
local_calib_scores_lower = calib_pred_values[topk_idxs, 0] - calib_true_values[topk_idxs]
local_calib_scores_upper = calib_true_values[topk_idxs] - calib_pred_values[topk_idxs,-1]

# Calcula los cuantiles qhat para ambos límites del intervalo predictivo
local_quantile_lower = np.quantile(local_calib_scores_lower, local_q_level, method='higher')
local_quantile_upper = np.quantile(local_calib_scores_upper, local_q_level, method='higher')

In [121]:
local_weight

tensor([0.7414, 0.6121, 0.8280,  ..., 0.7282, 0.6696, 0.6962])

In [122]:
global_weight

tensor([0.2586, 0.3879, 0.1720,  ..., 0.2718, 0.3304, 0.3038])

In [123]:
lamda = 1.1

# Calcula los cuantiles combinados
comb_quantile_lower = (local_weight * local_quantile_lower + global_weight * global_quantile_lower) * lamda
comb_quantile_upper = (local_weight * local_quantile_upper + global_weight * global_quantile_upper) * lamda

In [124]:
test_pred_lower = new_pred_values[:, 0] - comb_quantile_lower[:]
test_pred_upper = new_pred_values[:,-1] + comb_quantile_upper[:]
test_true_values = new_true_values

# Calcula la cobertura empírica y lo imprime
EC = empirical_coverage(test_pred_lower, test_pred_upper, test_true_values)
print(f"Cobertura empírica (para {(1-alpha)*100}% de confianza): {EC*100:>6.3f} %")

# Calcula el tamaño medio del intervalo de predicción y lo imprime
MIS = mean_interval_size(test_pred_lower, test_pred_upper)
print(f"Tamaño medio del intervalo: {MIS:>5.3f}")

Cobertura empírica (para 90.0% de confianza): 88.708 %
Tamaño medio del intervalo: 4.811
